In [147]:
#导入
import pandas as pd
pd.reset_option('display.float_format')
pd.options.display.float_format = '{:.2f}'.format
from openpyxl import load_workbook

df_ffca= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\FFCA.xlsx")
df_qv = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\QV.xlsx")
df_stock = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\stock.xlsx") 
df_stock.drop([0])
df_inventory = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\inventory.xlsx")
df_CS = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\CS_01.xlsx")
df_CP = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\CP_approved.xlsx")
df_movement = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\Movement.xlsx")
df_WC = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\Warehouse_Risk_Category.xlsx")
df_vessel = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\inventory2.xlsx")
df_invoice = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\invoice.xlsx")

In [148]:
input= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\input.xlsx")

In [150]:
#filter

cols_ffca_1 = ['Purchase Assignment ID', 'Inventory Status']
cols_input = ['assignment']

df_ffca_filter = pd.DataFrame(df_ffca, columns=cols_ffca_1)
df_input = pd.DataFrame(input, columns=cols_input)

assignment_filter = pd.merge(df_ffca_filter, df_input, left_on=["Purchase Assignment ID"], right_on=["assignment"], how= 'inner')
assignment_filter.columns = ['Purchase Assignment Reference', 'Inventory Status', 'Assignment']

def expected(a):
    return a == "Expected"
def instorage(a):
    return a == "In-Storage"
def intransit(a):
    return a == "In-Transit"
def delivered(a):
    return a == "Delivered"

df1 = assignment_filter.loc[assignment_filter['Inventory Status'].apply(expected)]
df2 = assignment_filter.loc[assignment_filter['Inventory Status'].apply(instorage)]
df3 = assignment_filter.loc[assignment_filter['Inventory Status'].apply(intransit)]
df4 = assignment_filter.loc[assignment_filter['Inventory Status'].apply(delivered)]

In [151]:
df1 = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\df2.xlsx")
df2 = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\df1.xlsx")


In [177]:
#prepayment
cols_ffca = ['Purchase Assignment ID', 'Commodity', 'Inventory ID','Counterparty of Purchase', \
    'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator']
cols_inventory = ['INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'EXPECTED RECEIPT DATE']
cols_stock = ['Inventory Id','Origin Country']

df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)

ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'outer')
inventory_stock = pd.merge(df_inventory_f1, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
final_prepayment = pd.merge(ffca_qv, inventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])

final_prepayment.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Settlement Valuation(USD)',  'Authorised Paid Quantity', 'Financed Quantity','Inventory Current Value(USD)',\
          'Purchase First Paid Date', 'Purchase Invoice ID', 'Purchase Invoice Amount(USD)','Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'INVENTORY ID', \
                      'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight', 'Purchase Expected Receipt Date', 'Inventory Id', 'Purchase Material Origin Country']

#增加列
final_prepayment['Purchase Covered Amount(USD)'] = '0'
final_prepayment['Purchase Dry Weight UOM'] = 'MT'
final_prepayment['Purchase Wet Weight UOM'] = 'MT'
final_prepayment['Receipt In Progress'] = 'N'
final_prepayment['Delivery In Progress'] = 'N'
final_prepayment['Authorised Paid Quantity'] = pd.to_numeric(final_prepayment['Authorised Paid Quantity'], errors='coerce')
final_prepayment['Financed Quantity'] = pd.to_numeric(final_prepayment['Financed Quantity'], errors='coerce')

final_prepayment.loc[:,"Financing Ratio"] = final_prepayment["Authorised Paid Quantity"]/final_prepayment["Financed Quantity"]

final_prepayment.loc[:, 'Purchase Paid Amount(USD)'] = final_prepayment['Purchase Invoice Amount(USD)']

#复制列
final_prepayment.loc[:, 'Eligible Dry Weight'] = final_prepayment['Purchase Dry Weight']
final_prepayment.loc[:, 'Eligible Wet Weight'] = final_prepayment['Purchase Wet Weight']

#删除列
final_prepayment.drop(columns=[ 'Assignment ID', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Inventory Id'])
#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty',  \
        'Purchase Commodity', \
             'Purchase Dry Weight', 'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Material Origin Country', \
                'Purchase Settlement Valuation(USD)', 'Purchase First Paid Date', 'Purchase Expected Receipt Date', 'Purchase Invoice ID', \
                    'Purchase Invoice Amount(USD)', 'Purchase Paid Amount(USD)', 'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio', \
                        'Purchase Covered Amount(USD)', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

final_prepayment = final_prepayment[order]

result1 = pd.merge(final_prepayment, df1, on=["Purchase Assignment Reference"])
# result1 = result1.drop_duplicates(['Purchase Assignment Reference'])

import numpy as np
test_p = result1.copy(deep=True)

test_p['Purchase Contractual Counterparty'] = np.nan
test_p['Purchase Settlement Valuation(USD)'] = np.nan
test_p['Authorised Paid Quantity'] = np.nan
test_p['Financed Quantity'] = np.nan
test_p['Financing Ratio'] = np.nan
test_p['Purchase Covered Amount(USD)'] = np.nan
test_p['Purchase Uncovered Amount(USD)'] = np.nan
test_p['Receipt In Progress'] = np.nan
test_p['Delivery In Progress'] = np.nan
test_p['Eligible Dry Weight'] = np.nan
test_p['Eligible Wet Weight'] = np.nan


test_q = result1.copy(deep=True)
test_q['Purchase Invoice ID'] = np.nan


test_f1 = pd.concat([test_p,test_q])
test_f1= test_f1.sort_values(by=['Purchase Assignment Reference'], ascending=False)

writer = pd.ExcelWriter(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\final_test.xlsx", engine='openpyxl')
book = load_workbook(writer.path)
writer.book = book
test_f1.to_excel(excel_writer=writer, sheet_name='prepayment')
writer.save()
writer.close()


In [181]:
#storage
#column filter
#column filter
cols_ffca = ['Purchase Assignment ID', 'Commodity', 'Inventory ID','Counterparty of Purchase', \
    'Purchase Settled Amount', 'Collateral Doc Type', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator', 'Collateral Doc Date', 'Location', 'Country']
cols_inventory = ['INVENTORY ID','COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'CITY']
cols_stock = ['Inventory Id','Origin Country']
cols_wc = ['LOCATION_DESCRIPTION', 'RISK_CATEGORY']

cols_ffca_blend = ['Purchase Assignment ID', 'Commodity', 'Inventory ID','Counterparty of Purchase', \
    'Purchase Settled Amount', 'Collateral Doc Type', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'QTY']

#NN
# cols_stock = ['Inventory Id','Origin Country', 'Inventory Quantity(Wet)', 'Delivery In Progress Qty', 'Receipt In Progress Qty (Wet)']

# df_inventory['INVENTORY ID'] = df_inventory['INVENTORY ID'].astype(str)
# df_stock['Inventory Id'] = df_stock['Inventory Id'].astype(str)
# df_ffca['Inventory ID'] = df_ffca['Inventory ID'].astype(str)

df_stock['Receipt In Progress Qty (Wet)'] = pd.to_numeric(df_stock['Receipt In Progress Qty (Wet)'], errors='coerce')
df_stock['Receipt In Progress Qty (Wet)'] = df_stock['Receipt In Progress Qty (Wet)'].fillna(0)


ndf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_WC_f1 = pd.DataFrame(df_WC, columns=cols_wc)
df_WC_f1['RISK_CATEGORY'] = df_WC_f1['RISK_CATEGORY'].map(lambda x: str(x)[13:])

#blend
bdf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca_blend)

def blend(a):
    return a == "Blend"

df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_nblend_inventory = df_inventory_f1.drop(row_list)

bffca_qv = pd.merge(bdf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
binventory_stock = pd.merge(df_blend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
bfinal_BB_WHS = pd.merge(bffca_qv, binventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
bfinal_BB_WHS = pd.merge(bfinal_BB_WHS,df_WC_f1, left_on = 'Location', right_on= 'LOCATION_DESCRIPTION', how='outer')


def function(a, b, c):
    return c * (1 + (a-b)/a)

bfinal_BB_WHS['Purchased Wet Weight']  = bfinal_BB_WHS.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_BB_WHS = bfinal_BB_WHS.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])

#改名
bfinal_BB_WHS.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Authorised Paid Quantity', \
        'Financed Quantity', 'Inventory Current Value(USD)', 'Purchase Dry Weight' , 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Warehouse Reference', 'Warehouse Country', \
                    'INVENTORY ID',  'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'Warehouse City', 'Inventory Id','Purchase Material Origin Country',\
                         'LOCATION_DESCRIPTION', 'Warehouse Category',  'Purchase Wet Weight']

bfinal_BB_WHS['Collateral Reference'] = None
bfinal_BB_WHS['Purchase Dry Weight UOM'] = 'MT'
bfinal_BB_WHS['Purchase Wet Weight UOM'] = 'MT'
bfinal_BB_WHS['Receipt In Progress'] = 'N'
bfinal_BB_WHS['Delivery In Progress'] = 'N'
# bfinal_BB_WHS['Receipt In Progress'] = bfinal_BB_WHS.apply(lambda x: function(x['Receipt In Progress Qty (Wet)'], x['Inventory Quantity(Wet)']), axis = 1)
# bfinal_BB_WHS['Delivery In Progress'] = bfinal_BB_WHS.apply(lambda x: function(x['Delivery In Progress Qty'], x['Inventory Quantity(Wet)']), axis = 1)

bfinal_BB_WHS['Authorised Paid Quantity'] = pd.to_numeric(bfinal_BB_WHS['Authorised Paid Quantity'], errors='coerce')
bfinal_BB_WHS['Financed Quantity'] = pd.to_numeric(bfinal_BB_WHS['Financed Quantity'], errors='coerce')
bfinal_BB_WHS.loc[:,"Financing Ratio"] = bfinal_BB_WHS["Authorised Paid Quantity"]/bfinal_BB_WHS["Financed Quantity"]

# bfinal_BB_WHS[['Authorized Paid Quantity', 'Financed Quantity']] = bfinal_BB_WHS[['Authorized Paid Quantity', 'Financed Quantity']].astype('float')
# bfinal_BB_WHS.loc[:,"Financing Ratio"] = bfinal_BB_WHS["Authorized Paid Quantity"]/bfinal_BB_WHS["Financed Quantity"]

#复制列
bfinal_BB_WHS.loc[:, 'Eligible Dry Weight'] = bfinal_BB_WHS['Purchase Dry Weight']
bfinal_BB_WHS.loc[:, 'Eligible Wet Weight'] = bfinal_BB_WHS['Purchase Wet Weight']

#删除列
# bfinal_BB_WHS.drop(columns=['Inventory Quantity(Wet)', 'Delivery In Progress Qty', 'Receipt In Progress Qty (Wet)', 'LOCATION_DESCRIPTION'])
bfinal_BB_WHS.drop(columns=[ 'LOCATION_DESCRIPTION'])
#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty', 'Collateral Document Type', 'Collateral Reference', 'COLLATERAL DOCUMENT DATE', \
        'Purchase Commodity', 'Purchase Material Origin Country', 'Warehouse Reference', 'Warehouse Category', 'Warehouse Country', 'Warehouse City', \
             'Purchase Dry Weight', \
                'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)',\
                    'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorised Paid Quantity', \
                        'Financed Quantity','Financing Ratio', 'Inventory Current Value(USD)']

bfinal_BB_WHS_test = bfinal_BB_WHS[order]


#non-blend

nffca_qv = pd.merge(ndf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
ninventory_stock = pd.merge(df_nblend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
nfinal_BB_WHS = pd.merge(nffca_qv, ninventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
nfinal_BB_WHS = pd.merge(nfinal_BB_WHS,df_WC_f1, left_on = 'Location', right_on= 'LOCATION_DESCRIPTION', how='outer')

#改名
nfinal_BB_WHS.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Authorised Paid Quantity', \
        'Financed Quantity', 'Inventory Current Value(USD)', 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Warehouse Reference', 'Warehouse Country', \
                    'INVENTORY ID',  'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight', 'Warehouse City', 'Inventory Id','Purchase Material Origin Country',\
                         'LOCATION_DESCRIPTION', 'Warehouse Category']


# #新增固定列
# def function(a,b):
#     if a == b:
#         return 'N'
#     elif a == 0:
#         return 'N'
#     else:
#         return 'Y'
        
nfinal_BB_WHS['Collateral Reference'] = None
nfinal_BB_WHS['Purchase Dry Weight UOM'] = 'MT'
nfinal_BB_WHS['Purchase Wet Weight UOM'] = 'MT'
nfinal_BB_WHS['Receipt In Progress'] = 'N'
nfinal_BB_WHS['Delivery In Progress'] = 'N'
# nfinal_BB_WHS['Receipt In Progress'] = nfinal_BB_WHS.apply(lambda x: function(x['Receipt In Progress Qty (Wet)'], x['Inventory Quantity(Wet)']), axis = 1)
# nfinal_BB_WHS['Delivery In Progress'] = nfinal_BB_WHS.apply(lambda x: function(x['Delivery In Progress Qty'], x['Inventory Quantity(Wet)']), axis = 1)

nfinal_BB_WHS['Authorised Paid Quantity'] = pd.to_numeric(nfinal_BB_WHS['Authorised Paid Quantity'], errors='coerce')
nfinal_BB_WHS['Financed Quantity'] = pd.to_numeric(nfinal_BB_WHS['Financed Quantity'], errors='coerce')
nfinal_BB_WHS.loc[:,"Financing Ratio"] = nfinal_BB_WHS["Authorised Paid Quantity"]/nfinal_BB_WHS["Financed Quantity"]



#复制列
nfinal_BB_WHS.loc[:, 'Eligible Dry Weight'] = nfinal_BB_WHS['Purchase Dry Weight']
nfinal_BB_WHS.loc[:, 'Eligible Wet Weight'] = nfinal_BB_WHS['Purchase Wet Weight']

#删除列
# nfinal_BB_WHS.drop(columns=['Inventory Quantity(Wet)', 'Delivery In Progress Qty', 'Receipt In Progress Qty (Wet)', 'LOCATION_DESCRIPTION'])
nfinal_BB_WHS.drop(columns=[ 'LOCATION_DESCRIPTION'])
#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty', 'Collateral Document Type', 'Collateral Reference', 'COLLATERAL DOCUMENT DATE', \
        'Purchase Commodity', 'Purchase Material Origin Country', 'Warehouse Reference', 'Warehouse Category', 'Warehouse Country', 'Warehouse City', \
             'Purchase Dry Weight', \
                'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)',\
                    'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorised Paid Quantity', \
                        'Financed Quantity','Financing Ratio', 'Inventory Current Value(USD)']

nfinal_BB_WHS_test = nfinal_BB_WHS[order]  

final_BB_WHS = pd.concat([nfinal_BB_WHS_test,bfinal_BB_WHS_test])


result2 = pd.merge(final_BB_WHS, df2, on=["Purchase Assignment Reference"], how='right')
result2 = result2.drop_duplicates(['Purchase Assignment Reference'])
writer = pd.ExcelWriter(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\final_test.xlsx", engine='openpyxl')
book = load_workbook(writer.path)
writer.book = book
result2.to_excel(excel_writer=writer, sheet_name='storage')
writer.save()
writer.close()

In [182]:
#transit
#column filter
cols_ffca = ['Purchase Assignment ID', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'QTY', \
    'Purchase Settled Amount', 'Collateral Doc Type', 'Financing Bank facility', 'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value']

cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator', 'Collateral Doc Date', 'Transport Mode']

cols_movement = ['ID', 'LOAD DATE','LOAD', 'DESTINATION', 'DESTINATION DATE']

cols_inventory = ['INVENTORY ID', 'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'MOVEMENT ID','RECEIPT INCOTERMS', 'DRY WEIGHT', 'WET WEIGHT']

cols_stock = ['Inventory Id','Origin Country']

cols_vessel = ['TRANSPORT', 'INVENTORY ID']


#转换数据类型

df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_vessel_f1 = pd.DataFrame(df_vessel, columns=cols_vessel)
df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_movement_f1 = pd.DataFrame(df_movement, columns=cols_movement)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_movement_f1['LOAD'] = df_movement_f1['LOAD'].astype(str)

#deliver detail
df_movement_f1.loc[:, 'Load Country'] = df_movement_f1['LOAD'].apply(lambda st: st[st.rfind("(")+1:st.rfind(")")])
df_movement_f1.loc[:, 'Delivery Country'] = df_movement_f1['DESTINATION'].apply(lambda st: st[st.rfind("(")+1:st.find(")")])

def blend(a):
    return a == "Blend"

df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_nblend_inventory = df_inventory_f1.drop(row_list)

ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'left')
bmovement_inventory = pd.merge(df_movement_f1, df_blend_inventory, left_on=["ID"], right_on=["MOVEMENT ID"])
bMIS = pd.merge(bmovement_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
bfinal_BB1 = pd.merge(ffca_qv, bMIS, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
bfinal_BB = pd.merge(bfinal_BB1, df_vessel_f1, on=['INVENTORY ID'])


def function(a, b, c):
    return c * (1 + (a-b)/a)

bfinal_BB['Purchase Wet Weight']  = bfinal_BB.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_BB = bfinal_BB.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])



#改名
bfinal_BB.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Dry Weight', 'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Financing Bank facility', 'Purchase Settled Amount', 'Authorised Paid Quantity', \
        'Financed Quantity', 'In Transit Current Value(USD)', 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Purchase Transport Mode', 'ID', \
                 'Load Date', 'Load Location', 'Delivery Location', 'Expected Delivery Date', 'Load Country', 'Destination Country', \
                    'INVENTORY ID', 'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'MOVEMENT ID', 'Purchase Incoterm', 'Inventory Id','Purchase Material Origin Country','Vessel Name', 'Purchase Wet Weight']


#新增固定列
bfinal_BB['Collateral Reference'] = None
bfinal_BB['Purchase Dry Weight UOM'] = 'MT'
bfinal_BB['Purchase Wet Weight UOM'] = 'MT'
bfinal_BB['Receipt In Progress'] = 'N'
bfinal_BB['Delivery In Progress'] = 'N'
# bfinal_BB['Receipt In Progress'] = bfinal_BB.apply(lambda x: function(x['Receipt In Progress Qty (Wet)'], x['Inventory Quantity(Wet)']), axis = 1)
# bfinal_BB['Delivery In Progress'] = bfinal_BB.apply(lambda x: function(x['Delivery In Progress Qty'], x['Inventory Quantity(Wet)']), axis = 1)
# bfinal_BB[['Authorized Paid Quantity', 'Financed Quantity']] = bfinal_BB[['Authorized Paid Quantity', 'Financed Quantity']].astype('float')
# bfinal_BB.loc[:,"Financing Ratio"] = bfinal_BB["Authorized Paid Quantity"]/bfinal_BB["Financed Quantity"]

bfinal_BB['Authorised Paid Quantity'] = pd.to_numeric(bfinal_BB['Authorised Paid Quantity'], errors='coerce')
bfinal_BB['Financed Quantity'] = pd.to_numeric(bfinal_BB['Financed Quantity'], errors='coerce')

bfinal_BB.loc[:,"Financing Ratio"] = bfinal_BB["Authorised Paid Quantity"]/bfinal_BB["Financed Quantity"]

#复制列
bfinal_BB.loc[:, 'Eligible Dry Weight'] = bfinal_BB['Purchase Dry Weight']
bfinal_BB.loc[:, 'Eligible Wet Weight'] = bfinal_BB['Purchase Wet Weight']
bfinal_BB.loc[:, 'BL Date'] = bfinal_BB['COLLATERAL DOCUMENT DATE']


#删除不要的列
bfinal_BB.drop(columns=['Financing Bank facility', 'Purchase Settled Amount', 'Assignment ID', 'ID', \
    'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Collateral Document Date', 'MOVEMENT ID'])

#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty', 'Collateral Document Type', 'Collateral Reference', 'COLLATERAL DOCUMENT DATE', \
        'Purchase Commodity', 'Purchase Material Origin Country', 'Purchase Incoterm', 'Purchase Transport Mode', 'Vessel Name', 'Load Date', \
            'Load Location', 'Load Country', 'BL Date', 'Expected Delivery Date', 'Delivery Location', 'Destination Country', 'Purchase Dry Weight', \
                'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)',\
                    'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorised Paid Quantity', \
                        'Financed Quantity','Financing Ratio', 'In Transit Current Value(USD)']
bfinal_BB_test = bfinal_BB[order]

movement_inventory = pd.merge(df_movement_f1, df_inventory_f1, left_on=["ID"], right_on=["MOVEMENT ID"], how='outer')
MIS = pd.merge(movement_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
final_BB1 = pd.merge(ffca_qv, MIS, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
final_BB = pd.merge(final_BB1, df_vessel_f1, on=['INVENTORY ID'])

# #新增固定列
# def function(a,b):
#     if a == b:
#         return 'N'
#     elif a == 0:
#         return 'N'
#     else:
#         return 'Y'

#改名
final_BB.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'QTY', 'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Financing Bank facility', 'Purchase Settled Amount', 'Authorised Paid Quantity', \
        'Financed Quantity', 'In Transit Current Value(USD)', 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Purchase Transport Mode', 'ID', \
                 'Load Date', 'Load Location', 'Delivery Location', 'Expected Delivery Date', 'Load Country', 'Destination Country', \
                    'INVENTORY ID', 'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'MOVEMENT ID', 'Purchase Incoterm', 'Purchase Dry Weight', 'Purchase Wet Weight','Inventory Id','Purchase Material Origin Country','Vessel Name']


#新增固定列
final_BB['Collateral Reference'] = None
final_BB['Purchase Dry Weight UOM'] = 'MT'
final_BB['Purchase Wet Weight UOM'] = 'MT'
final_BB['Receipt In Progress'] = 'N'
final_BB['Delivery In Progress'] = 'N'
# final_BB['Receipt In Progress'] = final_BB.apply(lambda x: function(x['Receipt In Progress Qty (Wet)'], x['Inventory Quantity(Wet)']), axis = 1)
# final_BB['Delivery In Progress'] = final_BB.apply(lambda x: function(x['Delivery In Progress Qty'], x['Inventory Quantity(Wet)']), axis = 1)
# final_BB[['Authorized Paid Quantity', 'Financed Quantity']] = final_BB[['Authorized Paid Quantity', 'Financed Quantity']].astype('float')
# final_BB.loc[:,"Financing Ratio"] = final_BB["Authorized Paid Quantity"]/final_BB["Financed Quantity"]

final_BB['Authorised Paid Quantity'] = pd.to_numeric(final_BB['Authorised Paid Quantity'], errors='coerce')
final_BB['Financed Quantity'] = pd.to_numeric(final_BB['Financed Quantity'], errors='coerce')

final_BB.loc[:,"Financing Ratio"] = final_BB["Authorised Paid Quantity"]/final_BB["Financed Quantity"]

#复制列
final_BB.loc[:, 'Eligible Dry Weight'] = final_BB['Purchase Dry Weight']
final_BB.loc[:, 'Eligible Wet Weight'] = final_BB['Purchase Wet Weight']
final_BB.loc[:, 'BL Date'] = final_BB['COLLATERAL DOCUMENT DATE']


#删除不要的列
final_BB.drop(columns=['QTY', 'Financing Bank facility', 'Purchase Settled Amount', 'Assignment ID', 'ID', \
    'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Collateral Document Date', 'MOVEMENT ID'])

#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty', 'Collateral Document Type', 'Collateral Reference', 'COLLATERAL DOCUMENT DATE', \
        'Purchase Commodity', 'Purchase Material Origin Country', 'Purchase Incoterm', 'Purchase Transport Mode', 'Vessel Name', 'Load Date', \
            'Load Location', 'Load Country', 'BL Date', 'Expected Delivery Date', 'Delivery Location', 'Destination Country', 'Purchase Dry Weight', \
                'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)',\
                    'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorised Paid Quantity', \
                        'Financed Quantity','Financing Ratio', 'In Transit Current Value(USD)']
final_BB_test = final_BB[order]
final_BB_test

final_BB = pd.concat([bfinal_BB_test,final_BB_test])

result3 = pd.merge(final_BB, df3, on=["Purchase Assignment Reference"], how='right')
result3 = result3.drop_duplicates(['Purchase Assignment Reference'])

writer = pd.ExcelWriter(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\final_test.xlsx", engine='openpyxl')
book = load_workbook(writer.path)
writer.book = book
result3.to_excel(excel_writer=writer, sheet_name='transit')
writer.save()
writer.close()

In [183]:
#sales
#sales

#column filter
cols_ffca = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
    'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
        'Sales Invoiced Amount', 'Sales Settled Amount']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator']
cols_inventory = ['INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'EXPECTED RECEIPT DATE']
cols_stock = ['Inventory Id','Origin Country']
cols_cs = ['ID', 'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank', 'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount']
cols_ffca_blend = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
    'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
        'Sales Invoiced Amount', 'Sales Settled Amount', 'QTY']
cols_invoice = ['Assignment Ref', 'Invoice Type', 'Invoice Date', 'Payment Expected Due Date', 'Days Overdue', 'Invoice No']

df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_CS_f1 = pd.DataFrame(df_CS, columns=cols_cs)
df_invoice_f1 = pd.DataFrame(df_invoice, columns=cols_invoice)
df_ffca_f1['Current Value'] = pd.to_numeric(df_ffca_f1['Current Value'], errors='coerce')
df_ffca_f1['Authorised Paid Quantity'] = pd.to_numeric(df_ffca_f1['Authorised Paid Quantity'], errors='coerce')
df_ffca_f1['Financed Quantity'] = pd.to_numeric(df_ffca_f1['Financed Quantity'], errors='coerce')


#sales
#delete nan sales
df_ffca_f1.dropna(subset=['Sales Assignment ID '], inplace=True)
# 一、先将字段拆分
df_CS_f1['Assignments']=df_CS['Assignments'].map(lambda x:x.split(','))
#二、然后直接调用explode()方法
df_CS_f1_explode = df_CS_f1.explode('Assignments')

def blend(a):
    return a == "Blend"

#blend
bdf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca_blend)
df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_nblend_inventory = df_inventory_f1.drop(row_list)

bffca_qv = pd.merge(bdf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
bffca_qv_cp = pd.merge(bffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
binventory_stock = pd.merge(df_blend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
FQC = pd.merge(bffca_qv_cp, df_CS_f1_explode, left_on=["Sales Assignment ID "], right_on=["Assignments"], how = 'outer')
bfinal_sales = pd.merge(FQC, binventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])

def function(a, b, c):
    return c * (1 + (a-b)/a)

bfinal_sales['Purchased Wet Weight']  = bfinal_sales.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_sales = bfinal_sales.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])


ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
ffca_qv_cp = pd.merge(ffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
inventory_stock = pd.merge(df_nblend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
nfinal_sales = pd.merge(FQC, inventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
nfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
       'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
       'Sales Invoice ID', 'Estimated Sales Valuation',
       'Purchase Settled Amount', 'Authorised Paid Quantity',
       'Financed Quantity', 'Current Value', 'Treasury Payment Date',
       'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
       'Sales Settled Amount', 'QTY', 'Assignment ID', 'Group Company',
       'Trade ID', 'Quota ID', 'Traffic Operator',
       'Sales Assignment Counterparty',
       'Sales Contractual Counterparty Open Account Approved ', 'ID',
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
       'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
       'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

order = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity','Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
       'Purchase Settled Amount', 'Authorised Paid Quantity','Financed Quantity', 'Current Value', 'Treasury Payment Date','Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount','Sales Settled Amount',  \
        'Assignment ID', 'Group Company','Trade ID', 'Quota ID', 'Traffic Operator','Sales Assignment Counterparty','Sales Contractual Counterparty Open Account Approved ', 'ID', \
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank','Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'QTY', 'Purchased Wet Weight',\
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

bfinal_sales = bfinal_sales[order]

bfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
       'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
       'Sales Invoice ID', 'Estimated Sales Valuation',
       'Purchase Settled Amount', 'Authorised Paid Quantity',
       'Financed Quantity', 'Current Value', 'Treasury Payment Date',
       'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
       'Sales Settled Amount', 'Assignment ID', 'Group Company', 'Trade ID',
       'Quota ID', 'Traffic Operator', 'Sales Assignment Counterparty',
       'Sales Contractual Counterparty Open Account Approved ', 'ID',
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
       'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
       'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

final_sales = pd.concat([nfinal_sales,bfinal_sales])

final_sales = pd.merge(final_sales, df_invoice_f1, left_on=["Sales Invoice ID"], right_on=["Invoice No"], how= 'inner')

final_sales.columns

final_sales['Current Value'] = pd.to_numeric(final_sales['Current Value'], errors='coerce')

#Comparison between invoice amount and current value


# def function1(a,b,c):
#     if 3 * (-b) >= a:
#         return (-b) - c
#     else:
#         return 0

def function1(a,b,c):
    if a == 'PROVISIONAL':
        return (-b) - c
    else:
        return 0

# def function2(a,b,c):
#     if 3 * (-b) < a:
#         return (-b) - c
#     else:
#         return 0

def function2(a,b,c):
    if a == 'FINAL':
        return (-b) - c
    else:
        return 0

final_sales['Unpaid Provisional Receivables(USD)'] = final_sales.apply(lambda x: function1(x['Invoice Type'], x['Sales Invoiced Amount'], x['Sales Settled Amount']), axis = 1)
final_sales['Unpaid Final Receivables(USD)'] = final_sales.apply(lambda x: function2(x['Invoice Type'], x['Sales Invoiced Amount'], x['Sales Settled Amount']), axis = 1) #wrong
final_sales.loc[:,"Uninvoiced Final Receivables(USD)"] = final_sales["Estimated Sales Valuation"] + final_sales["Sales Invoiced Amount"]

# final_sales.loc[:,"SCR Covered By LC (SCR-Sales Contractual Rights)"] = final_sales["Current Value"] - final_sales["Sales Invoiced Amount"]

#Comparison between invoice amount and LC current balance
def function3(a,b):
    if -a >= b:
        return b
    if -a < b:
        return -a

def function4(a,b):
    if -a < b:
        return b + a
    if -a >= b:
        return 0

def function5(a,b):
    if a >= b:
        return b
    else:
        return a

def function6(a):
    if a == "FINAL":
        return "100%"
    else:
        return "90%"


final_sales['Total Financeable Sales Invoices Covered by LC']  = final_sales.apply(lambda x: function3(x['Sales Invoiced Amount'], x['Balance Amount']), axis = 1)
final_sales['LC Cover Available to SCR (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function4(x['Sales Invoiced Amount'], x['Balance Amount']), axis = 1)
final_sales['Sales Provisional Percentage']  = final_sales.apply(lambda x: function6(x['Invoice Type']), axis = 1)
final_sales['SCR Covered By LC (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function5(x['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], x['Uninvoiced Final Receivables(USD)']), axis = 1)


import numpy as np

#adding 
final_sales['Sales Provisional Number'] = '1'
final_sales['Authorised Paid Quantity'] = pd.to_numeric(final_sales['Authorised Paid Quantity'], errors='coerce')
final_sales['Financed Quantity'] = pd.to_numeric(final_sales['Financed Quantity'], errors='coerce')
final_sales.loc[:,"Financing Ratio"] = final_sales["Authorised Paid Quantity"]/final_sales["Financed Quantity"]

final_sales['Cost Code ID'] = 'I01'
final_sales['Invoice Line Quantity'] = '0'
final_sales['Invoice Line Quantity UOM'] = 'MT'
final_sales['Purchase Dry Weight UOM'] = 'MT'
final_sales['Purchase Wet Weight UOM'] = 'MT'
final_sales['Receipt In Progress'] = 'N'
final_sales['Delivery In Progress'] = 'N'
final_sales['Summary Outstanding Receivables(USD)'] = final_sales['Unpaid Provisional Receivables(USD)'] + final_sales['Unpaid Final Receivables(USD)'] + final_sales['Uninvoiced Final Receivables(USD)']
final_sales['Sales Invoice Counterparty'] = final_sales['Counterparty of Sales']
final_sales['Eligible Dry Weight'] = final_sales['Purchase Dry Weight']
final_sales['Eligible Wet Weight'] = final_sales['Purchase Wet Weight']

def function7(a, b):
    if b == 0:
        return 0
    else:
        return -a/b
        
def function8(a):
    if a>1:
        return 1
    else:
        return a
        


final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function7(x['Estimated Sales Valuation'], x['Sales Invoiced Amount']), axis = 1)

final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function8(x['Sales Assignment Eligible Percentage']), axis = 1)

final_sales = final_sales.drop(columns=['Inventory ID', 'Current Value','Purchase Settled Amount', 'Purchase Invoice', 'Purchase Invoiced Amount', 'Current Value', 'Treasury Payment Date', \
    'Assignment ID', 'Sales Assignment Counterparty', 'Assignments', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', \
        'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country', 'QTY', 'Assignment Ref', 'Invoice No'], axis=1)


final_sales.columns = ['Purchase Assignment Reference', 'Sales Assignment Reference', 'Purchase Commodity',
       'Purchase Contractual Counterparty', 'Sales Assignment Counterparty', 'Sales Invoice ID',
       'Cost Amount(USD)', 'Authorised Paid Quantity',
       'Financed Quantity', 'Invoice Line Amount(USD)', 'Paid Amount',
       'Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Traffic Operator',
       'Sales Contractual Counterparty Open Account Approved', 'LC Reference', 'Sales Credit Security Type',
       'LC Opening Bank', 'LC Confirming Bank', 'LC Issuance Date', 'LC Expiry Date', 'LC Current Balance', 'LC Original Amount',
       'Purchase Dry Weight', 'Purchase Wet Weight', 
       'Sales Provisional Final', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Unpaid Provisional Receivables(USD)',
       'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)',
       'Total Financeable Sales Invoices Covered by LC',
       'LC Cover Available to SCR (SCR-Sales Contractual Rights)',
       'Sales Provisional Percentage',
       'SCR Covered By LC (SCR-Sales Contractual Rights)',
       'Sales Provisional Number', 'Financing Ratio', 'Cost Code ID',
       'Invoice Line Quantity', 'Invoice Line Quantity UOM',
       'Purchase Dry Weight UOM', 'Purchase Wet Weight UOM',
       'Receipt In Progress', 'Delivery In Progress',
       'Summary Outstanding Receivables(USD)', 'Sales Invoice Counterparty',
       'Eligible Dry Weight', 'Eligible Wet Weight',
       'Sales Assignment Eligible Percentage']


# final_sales.columns = ['Purchase Assignment Reference', 'Sales Assignment Reference', 'Purchase Commodity', 'Purchase Contractual Counterparty', \
#     'Sales Assignment Counterparty', 'Sales Invoice ID', 'Cost Amount(USD)', 'Authorised Paid Quantity', 'Financed Quantity', 'Invoice Line Amount(USD)', 'Paid Amount', 'Purchase Group Company Code', \
#         'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Sales Contractual Counterparty Open Account Approved', 'LC Reference', \
#             'Sales Credit Security Type','LC Opening Bank', 'LC Confirming Bank', 'LC Issuance Date', 'LC Expiry Date', 'LC Current Balance', 'LC Original Amount', \
#                 'Purchase Dry Weight', 'Purchase Wet Weight', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)',
#                     'Uninvoiced Final Receivables(USD)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Total Financeable Sales Invoices Covered by LC', \
#                         'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'Sales Provisional Final', 'Sales Provisional Percentage', \
#                             'Sales Provisional Number', 'Financing Ratio', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', \
#                                 'Cost Code ID', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Purchase Dry Weight UOM', 'Purchase Wet Weight UOM', 'Receipt In Progress', \
#                                     'Delivery In Progress', 'Summary Outstanding Receivables(USD)', 'Sales Invoice Counterparty','Eligible Dry Weight', 'Eligible Wet Weight', 'Sales Assignment Eligible Percentage']

order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Sales Assignment Reference','Purchase Traffic Operator', \
    'Purchase Contractual Counterparty', 'Purchase Commodity', 'Sales Assignment Counterparty', 'Sales Credit Security Type', 'LC Reference', 'LC Original Amount', \
        'LC Current Balance', 'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank', 'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC', \
            'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID', 'Sales Provisional Final', \
                'Sales Provisional Number', 'Sales Provisional Percentage', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Sales Invoice Counterparty', \
                    'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount', 'Sales Assignment Eligible Percentage', \
                        'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)', \
                            'Summary Outstanding Receivables(USD)', 'Sales Contractual Counterparty Open Account Approved', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', \
                                'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

final_sales_test = final_sales[order]

result4 = pd.merge(final_sales_test, df4, on=["Purchase Assignment Reference"], how='right')
result4 = result4.drop_duplicates(['Purchase Assignment Reference'])

import numpy as np
test_p = result4.copy(deep=True)

test_p['Purchase Contractual Counterparty'] = np.nan
test_p['Sales Credit Security Type'] = np.nan
test_p['LC Reference'] = np.nan
test_p['LC Original Amount'] = np.nan
test_p['LC Current Balance'] = np.nan
test_p['LC Issuance Date'] = np.nan
test_p['LC Expiry Date'] = np.nan
test_p['LC Opening Bank'] = np.nan
test_p['LC Confirming Bank'] = np.nan
test_p['Total Financeable Sales Invoices Covered by LC'] = np.nan
test_p['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = np.nan
test_p['SCR Covered By LC (SCR-Sales Contractual Rights)'] = np.nan
test_p['Cost Amount(USD)'] = np.nan
test_p['Authorised Paid Quantity'] = np.nan
test_p['Financed Quantity'] = np.nan
test_p['Financing Ratio'] = np.nan
test_p['Uninvoiced Final Receivables(USD)'] = np.nan
test_p['Summary Outstanding Receivables(USD)'] = np.nan
test_p['Sales Contractual Counterparty Open Account Approved '] = np.nan
test_p['Purchase Dry Weight'] = np.nan
test_p['Purchase Dry Weight UOM'] = np.nan
test_p['Purchase Wet Weight'] = np.nan
test_p['Purchase Wet Weight UOM'] = np.nan
test_p['Receipt In Progress'] = np.nan
test_p['Delivery In Progress'] = np.nan
test_p['Eligible Dry Weight'] = np.nan
test_p['Eligible Wet Weight'] = np.nan

test_q = result4.copy(deep=True)
test_q['Sales Invoice ID'] = np.nan
test_q['Sales Provisional Final '] = np.nan
test_q['Sales Provisional Number'] = np.nan
test_q['Sales Provisional Percentage'] = np.nan
test_q['Sales Invoice Date'] = np.nan
test_q['Sales Invoice Due Date'] = np.nan
test_q['Sales Invoice Days Overdue'] = np.nan
test_q['Sales Invoice Counterparty'] = np.nan
test_q['Cost Code ID'] = np.nan
test_q['Invoice Line Quantity'] = np.nan
test_q['Invoice Line Quantity UOM'] = np.nan

test_f2 = pd.concat([test_p,test_q])
test_f2 = test_f2.sort_values(by=['Purchase Assignment Reference'], ascending=False)

writer = pd.ExcelWriter(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\final_test.xlsx", engine='openpyxl')
book = load_workbook(writer.path)
writer.book = book
test_f2.to_excel(excel_writer=writer, sheet_name='sales_receivables')
writer.save()
writer.close()

